In [1]:
import pandas as pd
import numpy as np
import xlrd
import glob
import os

import time

In [2]:
# this script parses important insurer information from an xlsx file 
# original data can be found here: https://www.cms.gov/apps/mlr/mlr-search.aspx

# author: Grace Guan
# date created: 11/24/17

In [3]:
ycoordlabels = ["1HealthInsuranceINDIVIDUALTotalasof12/31/17","2HealthInsuranceINDIVIDUALTotalasof3/31/18"
,"3HealthInsuranceINDIVIDUALDualContracts(IncludedinTotalasof3/31/18)","4HealthInsuranceINDIVIDUALDeferredPY1(Add)"
,"5HealthInsuranceINDIVIDUALDeferredCY(Subtract)"
,"6HealthInsuranceSMALLGROUPTotalasof12/31/17","7HealthInsuranceSMALLGROUPTotalasof3/31/18"
,"8HealthInsuranceSMALLGROUPDualContracts(IncludedinTotalasof3/31/18)","9HealthInsuranceSMALLGROUPDeferredPY1(Add)"
,"10HealthInsuranceSMALLGROUPDeferredCY(Subtract)",
"11HealthInsuranceLARGEGROUPTotalasof12/31/17","12HealthInsuranceLARGEGROUPTotalasof3/31/18"
,"13HealthInsuranceLARGEGROUPDualContracts(IncludedinTotalasof3/31/18)","14HealthInsuranceLARGEGROUPDeferredPY1(Add)"
,"15HealthInsuranceLARGEGROUPDeferredCY(Subtract)","16Mini-MedINDIVIDUALTotalasof12/31/17"
,"17Mini-MedINDIVIDUALTotalasof3/31/18","18Mini-MedINDIVIDUALDualContracts(IncludedinTotalasof3/31/18)"
,"19Mini-MedSMALLGROUPTotalasof12/31/17","20Mini-MedSMALLGROUPTotalasof3/31/18"
,"21Mini-MedSMALLGROUPDualContracts(IncludedinTotalasof3/31/18)","22Mini-MedLARGEGROUPTotalasof12/31/17"
,"23Mini-MedLARGEGROUPTotalasof3/31/18","24Mini-MedLARGEGROUPDualContracts(IncludedinTotalasof3/31/18)"
,"25ExpatSMALLGROUPTotalasof12/31/17","26ExpatSMALLGROUPTotalasof3/31/18"
,"27ExpatSMALLGROUPDualContracts(IncludedinTotalasof3/31/18)","28ExpatSMALLGROUPDeferredPY1(Add)"
,"29ExpatSMALLGROUPDeferredCY(Subtract)","30ExpatLARGEGROUPTotalasof12/31/17","31ExpatLARGEGROUPTotalasof3/31/18"
,"32ExpatLARGEGROUPDualContracts(IncludedinTotalasof3/31/18)","33ExpatLARGEGROUPDeferredPY1(Add)"
,"34ExpatLARGEGROUPDeferredCY(Subtract)","35StudentHealthINDIVIDUALTotalasof12/31/17"
,"36StudentHealthINDIVIDUALTotalasof3/31/18","37StudentHealthINDIVIDUALDualContracts(IncludedinTotalasof3/31/18)"
,"38StudentHealthINDIVIDUALDeferredPY1(Add)","39StudentHealthINDIVIDUALDeferredCY(Subtract)"
,"40GovernmentProgramPlansTotalasof12/31/17","41OtherHealthBusinessTotalasof12/31/17"
,"42Aggregate2%RuleTotalasof12/31/17","43UninsuredPlansTotalasof12/31/17","44GrandTotalTotalasof12/31/17"]
    
xcoordlabels1 = ["1.Premium",
"1.1Totaldirectpremiumearned",
"1.2Federalhighriskpools",
"1.3Statehighriskpools",
"1.4Netassumedlesscededreinsurancepremiumearned(excludeamountsalreadyreportedinLine1.1)",
"1.5OtheradjustmentsduetoMLRcalculations-premium",
"1.6Riskrevenue",
"2.Claims",
"2.1Totalincurredclaims(MLRFormPart2Line2.16)",
"2.2Prescriptiondrugs(informationalonly;alreadyincludedintotalincurredclaimsabove)",
"2.3Pharmaceuticalrebates(informationalonly;alreadyexcludedfromtotalincurredclaimsabove)",
"2.4Statestoplossmarketstabilizationandclaim/censusbasedassessments(informationalonly;alreadyexcludedfromtotalincurredclaimsabove)",
"2.5Netassumedlesscededclaimsincurred(excludeamountsalreadyreportedinLine2.1)",
"2.6OtheradjustmentsduetoMLRcalculations–claimsincurred",
"2.7Rebatespaid",
"2.8EstimatedrebatesunpaidattheendofthepreviousMLRreportingyear",
"2.9EstimatedrebatesunpaidattheendoftheMLRreportingyear",
"2.10Fee-for-serviceandco-payrevenue(netofexpenses)",
"2.11Allowablefraudreductionexpenses(MLRFormPart2Line2.17)",
"3.FederalandStateTaxesandLicensingorRegulatoryFees",
"3.1FederaltaxesandassessmentsincurredbythereportingissuerduringtheMLRreportingyear",
"3.1aFederalincometaxesdeductiblefrompremiuminMLRcalculations",
"3.1bPatientCenteredOutcomesResearchInstitute(PCORI)Fee",
"3.1cAffordableCareActsection9010Fee",
"3.1dOtherFederalTaxesandassessmentsdeductiblefrompremium",
"3.2StateinsurancepremiumandothertaxesincurredbythereportingissuerduringtheMLRreportingyear(deductiblefrompremiuminMLRcalculation)",
"3.2aStateincomeexcisebusinessandothertaxes",
"3.2bStatepremiumtaxes",
"3.2cCommunitybenefitexpendituresdeductiblefrompremiuminMLRcalculations",
"3.3Regulatoryauthoritylicensesandfees",
"3.3aFederalTransitionalReinsuranceProgramcontributions",
"3.3bOtherFederalandStateregulatoryauthoritylicensesandfees",
"4.HealthCareQualityImprovementExpensesIncurred",
"4.1Improvehealthoutcomes",
"4.2Activitiestopreventhospitalreadmission",
"4.3Improvepatientsafetyandreducemedicalerrors",
"4.4Wellnessandhealthpromotionactivities",
"4.5Healthinformationtechnologyexpensesrelatedtoimprovinghealthcarequality",
"4.6 Total allowable quality improvement expenses",
"5.Non-ClaimsCosts",
"5.1CostcontainmentexpensesnotincludedinqualityimprovementexpensesinSection4",
"5.2Allotherclaimsadjustmentexpenses",
"5.3Directsalessalariesandbenefits",
"5.4Agentsandbrokersfeesandcommissions",
"5.5Othertaxes",
"5.5aTaxesandassessments(excludeamountsreportedinSection3orLine9)",
"5.5bFinesandpenaltiesofregulatoryauthorities(excludeamountsreportedinLine3.3)",
"5.5c Federal and State employment taxes and assessments",
"5.6Othergeneralandadministrativeexpenses",
"5.7Communitybenefitexpenditures(informationalonly;includeamountsreportedinLines3.2cand5.6)",
"6.Incomefromfeesofuninsuredplans",
"7.OtherIndicatorsorinformation",
"7.1Numberofpolicies/certificates",
"7.2Numberofcoveredlives",
"7.3Numberofgroups",
"7.4Membermonths",
"7.5Numberoflife-years",
"8.Netinvestmentincomeandothergain/(loss)",
"9.OtherFederalincometaxes(excludetaxesonLines3.1a-d)"]

xcoordlabels2 = ["1.Premium",
"1.1Directpremiumwritten",
"1.2Unearnedpremiumprioryear",
"1.3UnearnedpremiumMLRReportingyear",
"1.4Experienceratingrefunds(ratecredits)paid",
"1.4aExperienceratingrefundswithallincurreddatespaidintheMLRreportingyear",
"1.4bExperienceratingrefundsassociatedwithpremiumearnedonlyinthereportingyearandpaidthrough3/31ofthefollowingyear",
"1.5Reserveforexperienceratingrefunds(ratecredits)MLRReportingyear",
"1.6Reserveforexperienceratingrefunds(ratecredits)prioryear",
"1.7Premiumbalanceswrittenoff",
"1.8Groupconversioncharges",
"1.9FederalTransitionalReinsuranceProgrampaymentsexpectedfromHHS(asindicatedbyHHSasof6/30)",
"1.10FederalRiskAdjustmentProgramnetpaymentsexpectedfromHHS/(chargespayabletoHHS)(asindicatedbyHHSasof6/30)",
"1.11FederalRiskCorridorsProgramnetpayments/(charges)",
"1.12Premiumcededunder100%reinsurance(informationalonly;alreadyexcludedfromLines1.1-1.11)",
"1.13Premiumassumedunder100%reinsurance(informationalonly;alreadyincludedinLines1.1-1.11)",
"1.14AdvancepaymentsofthepremiumtaxcreditreceivedfromHHS(informationalonly;alreadyincludedinLines1.1-1.11)",
"2.Claims",
"2.1ClaimsPaid",
"2.1aClaimspaidduringtheMLRreportingyearregardlessofincurreddate",
"2.1bClaimsincurredonlyduringtheMLRreportingyearpaidthrough3/31ofthefollowingyear",
"2.2Directclaimliability",
"2.2aLiabilityasof12/31ofMLRreportingyearforallclaimsregardlessofincurreddate",
"2.2bLiabilityforclaimsincurredonlyduringtheMLRreportingyearcalculatedasof3/31ofthefollowingyear",
"2.3Directclaimliabilityprioryear",
"2.4Directclaimreserves",
"2.4aReservesasof12/31ofMLRreportingyearforallclaimsregardlessofincurreddate",
"2.4bReservesforclaimsincurredonlyduringtheMLRreportingyearcalculatedasof3/31ofthefollowingyear",
"2.5Directclaimreservesprioryear",
"2.6Directcontractreserves",
"2.6aDirectcontractreserves12/31column",
"2.6bDirectcontractreserves3/31dualcontractdeferredcolumns",
"2.7Directcontractreservesprioryear",
"2.8Experienceratingrefunds(ratecredits)paid",
"2.8aExperienceratingrefundswithallincurreddatespaidintheMLRreportingyear",
"2.8bExperienceratingrefundsassociatedwithpremiumearnedonlyinthereportingyearandpaidthrough3/31ofthefollowingyear",
"2.9Reserveforexperienceratingrefunds(ratecredits)",
"2.9aReservedinMLRreportingyearregardlessofincurreddate",
"2.9bReservesspecifictotheMLRreportingyearthrough3/31ofthefollowingyear",
"2.10Reserveforexperienceratingrefunds(ratecredits)prioryear",
"2.11Incurredmedicalincentivepoolandbonuses",
"2.11aPaidmedicalincentivepoolsandbonusesMLRReportingyear",
"2.11bAccruedmedicalincentivepoolsandbonusesMLRReportingyear",
"2.11cAccruedmedicalincentivepoolsandbonusesprioryear",
"2.12Nethealthcarereceivables",
"2.12aHealthcarereceivablesMLRReportingyear",
"2.12bHealthcarereceivablesprioryear",
"2.13Contingentbenefitandlawsuitreserves",
"2.14Groupconversioncharges",
"2.15Blendedrateadjustment",
"2.16Totalincurredclaims",
"2.17Allowablefraudreductionexpense(thesmallerofLines2.17aor2.17b)",
"2.17aTotalfraudreductionexpense",
"2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.1",
"2.18Advancepaymentsofcost-sharingreductions"]

xcoordspart0 = ["CompanyName:",
"GroupAffiliation:",
"FederalEIN:",
"A.M.BestNumber:",
"NAICGroupCode:",
"NAICCompanyCode:",
"DBA/MarketingName:",
"HIOSIssuerID:",
"BusinessintheStateof:",
"DomiciliaryState:",
"Address:",
"FederalTaxExempt:",
"Marketplace:",
"MergeMarkets-Ind/SmGrp:",
"Not-For-Profit:",
"MLRReportingYear:"]


labels = list()
    
for a in range(0, 16): # parse sheet 0
    labels.append(xcoordspart0[a])

for b in range(0, 58): # parse sheet 1
    for c in range(0, 44):
        labels.append(xcoordlabels1[b] + ycoordlabels[c])

for d in range(0, 54): # parse sheet 2
    for e in range (0, 44):
        labels.append(xcoordlabels2[d] + ycoordlabels[e])
        
#print(labels)

In [4]:
def parse_input(df17, filename, xls):
    
    sheet0 = xls.parse(0)
      
    # 2014, 2015, and 2016 format
    if sheet0.iloc[2,0] != 'Company Name:': # should say "Company Name:"
        return

    reporting_year = sheet0.iloc[17,1]

    sheet1 = xls.parse(1)
    sheet2 = xls.parse(2)
    
    data = list()
    
    for a in range(2, 18): # parse sheet 0
        data.append(sheet0.iloc[a, 1])
    
    for b in range(2, 60): # parse sheet 1
        for c in range(2, 46):
            data.append(sheet1.iloc[b, c])
    
    for d in range(2, 56): # parse sheet 2
        for e in range (2, 46):
            data.append(sheet2.iloc[d, e])

    df17.loc[df17.shape[0]] =  data

In [7]:
# create the dataframe

t0 = time.time()

errorcount = 0

mydf17 = pd.DataFrame(columns=labels)

os.chdir('/Users/GraceG/Desktop/2018Project/Create2018Dataset/2018RawData') #2016_data_extracted
FileList = glob.glob('*.xls*')

FailedFileList = list()

i = 0
for File in FileList:
    try:
        myxls = pd.ExcelFile(File)
        parse_input(mydf17, File, myxls)
    except xlrd.XLRDError:
        print(File + " failure")
        errorcount = errorcount + 1
        FailedFileList.append(File)
        pass
    if i % 100 == 0:
        tx = time.time()
        print(str(i) + " files completed in " + str(tx-t0) + "s")
    i += 1


print("There were " + str(errorcount) + " encrypted files.")
mydf17.head()
mydf17.tail()


0 files completed in 1.8347148895263672s
100 files completed in 185.0135359764099s
MLR_Template_Florida copy 15.xlsx failure
MLR_Template_Massachusetts copy 14.xlsx failure
200 files completed in 358.0772190093994s
300 files completed in 559.2119948863983s
MLR_Template_Virginia copy 17.xlsx failure
400 files completed in 777.2114162445068s
MLR_Template_California copy 12.xlsx failure
500 files completed in 1018.0589809417725s
MLR_Template_Louisiana copy 13.xlsx failure
600 files completed in 1279.0316801071167s
700 files completed in 1573.7488050460815s
800 files completed in 1886.0547468662262s
900 files completed in 2243.5975029468536s
MLR_Template_Grand_Total copy 54.xlsx failure
1000 files completed in 2603.589485168457s
1100 files completed in 3032.901125907898s
1200 files completed in 3525.6778268814087s
MLR_Template_Grand_Total copy 305.xlsx failure
1300 files completed in 4045.4220809936523s
1400 files completed in 4673.916353225708s
MLR_Template_Pennsylvania copy 16.xlsx failu

,CompanyName:,GroupAffiliation:,FederalEIN:,A.M.BestNumber:,NAICGroupCode:,NAICCompanyCode:,DBA/MarketingName:,HIOSIssuerID:,BusinessintheStateof:,DomiciliaryState:,...,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.135StudentHealthINDIVIDUALTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.136StudentHealthINDIVIDUALTotalasof3/31/18,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.137StudentHealthINDIVIDUALDualContracts(IncludedinTotalasof3/31/18),2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.138StudentHealthINDIVIDUALDeferredPY1(Add),2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.139StudentHealthINDIVIDUALDeferredCY(Subtract),2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.140GovernmentProgramPlansTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.141OtherHealthBusinessTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.142Aggregate2%RuleTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.143UninsuredPlansTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.144GrandTotalTotalasof12/31/17
2132,"Neighborhood Health Partnership, Inc.",UNITEDHEALTH GRP,650996107,064001,00707,95123,"Neighborhood Health Partnership, Inc.",80779,Florida,Florida,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN
2133,Illinois Mutual Life Insurance Company,Illinois Mutual Life Insurance Company,370344290,NaN,NaN,64580,Illinois Mutual Life Insurance Company,99219,Iowa,Illinois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2134,Trustmark Insurance Company,Trustmark Insurance Company,360792925,NaN,NaN,61425,Trustmark Insurance Company,88223,Utah,Illinois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2135,New York Life Insurance Company,New York Life Grp,135582869,006820,00826,66915,New York Life Insurance Company,87585,Virginia,New York,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2136,Banner Health and Aetna Health Insurance Company,AETNA GRP,815281115,NaN,1,16058,Banner Health and Aetna Health Insurance Company,NaN,Grand Total,Arizona,...,0,0,0,0,0,0,0,0,0,NaN


In [8]:

t1 = time.time()

print(t1-t0)

9570.281894922256


In [9]:
mydf17.to_csv("2018-all.csv")

In [10]:
print(FailedFileList)

['MLR_Template_Florida copy 15.xlsx', 'MLR_Template_Massachusetts copy 14.xlsx', 'MLR_Template_Virginia copy 17.xlsx', 'MLR_Template_California copy 12.xlsx', 'MLR_Template_Louisiana copy 13.xlsx', 'MLR_Template_Grand_Total copy 54.xlsx', 'MLR_Template_Grand_Total copy 305.xlsx', 'MLR_Template_Pennsylvania copy 16.xlsx', 'MLR_Template_Grand_Total copy 133.xlsx', 'MLR_Template_Michigan copy 21.xlsx', 'MLR_Template_Wisconsin copy 51.xlsx']


In [12]:
for File in FailedFileList:
    try:
        myxls = pd.ExcelFile(File)
        parse_input(mydf17, File, myxls)
    except xlrd.XLRDError:
        print(File + " failure")
        errorcount = errorcount + 1
        FailedFileList.append(File)
        pass
    if i % 100 == 0:
        tx = time.time()
        print(str(i) + " files completed in " + str(tx-t0) + "s")
    i += 1

In [13]:
mydf17.to_csv("2018-all-2.csv")

In [14]:
mydf17.tail()

,CompanyName:,GroupAffiliation:,FederalEIN:,A.M.BestNumber:,NAICGroupCode:,NAICCompanyCode:,DBA/MarketingName:,HIOSIssuerID:,BusinessintheStateof:,DomiciliaryState:,...,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.135StudentHealthINDIVIDUALTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.136StudentHealthINDIVIDUALTotalasof3/31/18,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.137StudentHealthINDIVIDUALDualContracts(IncludedinTotalasof3/31/18),2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.138StudentHealthINDIVIDUALDeferredPY1(Add),2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.139StudentHealthINDIVIDUALDeferredCY(Subtract),2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.140GovernmentProgramPlansTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.141OtherHealthBusinessTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.142Aggregate2%RuleTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.143UninsuredPlansTotalasof12/31/17,2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.144GrandTotalTotalasof12/31/17
2143,"Trilogy Health Insurance, Inc.",NaN,205330218,NaN,NaN,12834,"Trilogy Health Insurance, Inc.",NaN,Grand Total,Wisconsin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2144,Hartford Life & Accident Insurance Company,HARTFORD FIRE & CAS GRP,060838648,007285,00091,70815,Hartford Life & Accident Insurance Company,69150,Pennsylvania,Connecticut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2145,Hartford Life & Accident Insurance Company,HARTFORD FIRE & CAS GRP,060838648,007285,00091,70815,Hartford Life & Accident Insurance Company,NaN,Grand Total,Connecticut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2146,Hartford Life & Accident Insurance Company,HARTFORD FIRE & CAS GRP,060838648,007285,00091,70815,Hartford Life & Accident Insurance Company,30601,Michigan,Connecticut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2147,"Trilogy Health Insurance, Inc.",NaN,205330218,NaN,NaN,12834,"Trilogy Health Insurance, Inc.",18196,Wisconsin,Wisconsin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
